# Pyspark local mode

Auteur : Biard Antoine


Date : 11/10/2020

Dans ce notebook, nous présentons comment lancer un cluster pyspark en mode local, et présentons les helpers du repository.

## TLDR

Utiliser simplement :

In [1]:
from utils.spark import get_spark_sql_context, get_spark_context

sc = get_spark_context()
print(sc)
sql_context = get_spark_sql_context()
print(sql_context)


<SparkContext master=local[4] appName=notebooks>


In [2]:
sc.stop()

## Création d'un SparkContext

Source : [guide](https://docs.faculty.ai/how_to/spark/local_spark.html) de [Faculty.ai](faculty.ai).

D'abord nous précisons à Pyspark quel python utiliser.

In [3]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

Création d'une configuration en mode local.

In [4]:
import pyspark

number_cores = 8
memory_gb = 24

conf = (
    pyspark.SparkConf()
        .setMaster('local[{}]'.format(number_cores))
        .set('spark.driver.memory', '{}g'.format(memory_gb))
)

Création d'un contexte spark à partir de la configuration.

In [5]:
sc = pyspark.SparkContext(conf=conf)

Il ne peut y avoir qu'un seul context par configuration.

In [6]:
try:
    sc_same_conf = pyspark.SparkContext(conf=conf)
except ValueError as ve:
    print("Le chargement d'un nouveau context fails, l'erreur renvoyée par pyspark : ")
    print()
    print(ve)
    print()

Le chargement d'un nouveau context fails, l'erreur renvoyée par pyspark : 

Cannot run multiple SparkContexts at once; existing SparkContext(app=notebooks, master=local[8]) created by __init__ at <ipython-input-5-c9f39e965e09>:1 



`pyspark.SparkContext` possède une méthode de classe `getOrCreate` permettant de récupérer un context à partir d'une configuration sans ce problème.

L'implémentation derrière est faite sous la forme d'un [singleton](https://github.com/apache/spark/blob/master/python/pyspark/context.py#L364).

Id python du premier SparkContext :

In [7]:
id(sc)

140104062323152

Id python de `sc._active_spark_context` :

In [8]:
id(sc._active_spark_context)

140104062323152

Id python du SparkContext en utilisant `getOrCreate` :

In [9]:
sc_from_get_or_create =  pyspark.SparkContext.getOrCreate(conf)

id(sc_from_get_or_create)

140104062323152

Tous ces objets sont les mêmes.

## Fonctions dans `utils.spark`

Afin d'éviter d'écrire toutes ces lignes, le module `utils.spark` contient quelques helpers.

Le premier permet de récupérer le spark context actif :

In [10]:
from utils.spark import get_spark_context

sc_1 = get_spark_context()
print(sc_1)
print(id(sc_1))
sc_2 = get_spark_context()
print(sc_2)
print(id(sc_2))

<SparkContext master=local[8] appName=notebooks>
140104062323152
<SparkContext master=local[8] appName=notebooks>
140104062323152


Le second un sql context de la même manière :

In [11]:
from utils.spark import get_spark_sql_context

sqc = get_spark_sql_context()
sqc